In [13]:
# ! pip install tiktoken
# ! pip install --upgrade tiktoken
# from tiktoken import Tokenizer

In [760]:
import os
import re
import ast
import pandas as pd
import json
from IPython.display import display, HTML, Code
from IPython.display import clear_output
# from tiktoken import Tokenizer, TokenCount
# from tiktoken.models import Model

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 50)


C:\Users\N\AppData\Local\Temp\ipykernel_6204\26902468.py:11: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [173]:
import Tweet_Geo_constants as constants
import helper
# import LLM_Geo_kernel.Solution as Solution

from Tweet_Geo_kernel import Localization

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load state_abbreviation

In [552]:
State_abbr_CSV = r'https://raw.githubusercontent.com/gladcolor/Tweet_Geo/master/state_abbre.csv'
state_abbr_df = pd.read_csv(State_abbr_CSV)
state_abbr_df['State'] = state_abbr_df['State'].str.strip()
state_abbr_df['state_US'] = state_abbr_df['State'].str.title() + "; US" 
state_abbr_df.head()

,State,Abbreviation,state_US
0,ALABAMA,AL,Alabama; US
1,ALASKA,AK,Alaska; US
2,AMERICAN SAMOA,AS,American Samoa; US
3,ARIZONA,AZ,Arizona; US
4,ARKANSAS,AR,Arkansas; US


In [553]:
# convert two column values into a dictionary: the key is column "State", the value is the column "Abbreviation"
state_abbr_dict = state_abbr_df.set_index('State')['Abbreviation'].to_dict()
# state_abbr_dict

# Load tweet user information

In [879]:
# user_CSV = r'.\user_500.csv'
user_CSV = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Tweet_geo\usall_1m.csv'

# raw_user_df = pd.read_csv(user_CSV).fillna("")

# format the column names
raw_user_df['description'] = raw_user_df['userDescription'] 
raw_user_df['location'] = raw_user_df['userLocation'] 
raw_user_df['location'] = raw_user_df['location'].str.replace("\t ", "").replace("\n", "")

raw_user_df['id'] = raw_user_df['userID'] 

used_cols=['id', 'description', 'location']
raw_user_df = raw_user_df.fillna("")
raw_user_df['description'] = raw_user_df['description'].apply(helper.clean_tweet)
raw_user_df['location'] = raw_user_df['location'].apply(helper.clean_tweet)


raw_user_df['user_city'] = raw_user_df['location'].str.split(';').str[-2].str.strip().str.title()
raw_user_df['user_state'] = raw_user_df['location'].str.split(';').str[-1].str.strip().str.upper() 
raw_user_df['user_state'] = raw_user_df['user_state'].replace(state_abbr_dict)
raw_user_df['user_state'] = raw_user_df['user_state'].replace({"UNITED STATES": "US", "United States of America": "US", "USA": "US"})
# raw_user_df['user_state'] = raw_user_df['user_state'].replace({"USA": "US"})



raw_user_df['user_city_state'] = raw_user_df['user_city'] + '; ' + raw_user_df['user_state']


raw_user_df#[used_cols]

,userID,date,userLocation,userDescription,description,location,id,user_city,user_state,user_city_state
0,1064238776,2017-02-02 11:33:38,,IXVIIIXVII fly high babygirl,IXVIIIXVII fly high babygirl,,1064238776,NaN,,NaN
1,479920728,2017-02-02 11:33:39,where im meant to be,,,where im meant to be,479920728,NaN,WHERE IM MEANT TO BE,NaN
2,1596576888,2017-02-02 11:33:39,San Jacinto; CA,S'il vous plat; don't you hate,S'il vous plat; don't you hate,San Jacinto; CA,1596576888,San Jacinto,CA,San Jacinto; CA
3,748510143487279105,2017-02-02 11:33:39,67th Langley,,,67th Langley,748510143487279105,NaN,67TH LANGLEY,NaN
4,17754800,2017-02-02 11:33:39,,pioneer sound // delta & dawn,pioneer sound // delta & dawn,,17754800,NaN,,NaN
...,...,...,...,...,...,...,...,...,...,...
999994,242075925,2019-01-04 23:03:12,,alani,alani,,242075925,NaN,,NaN
999995,3517448480,2019-01-04 23:03:12,Florida; USA,Independent.Supports equality and gun control. Racism makes me sick; we are bigger and better than this. 1 tribe yall Trump is an epic scumbag Idiot.,Independent.Supports equality and gun control. Racism makes me sick; we are bigger and better than this. 1 tribe yall Trump is an epic scumbag Idiot.,Florida; USA,3517448480,Florida,US,Florida; US
999996,724096087,2019-01-04 23:03:12,Fayetteville; NC,@uncfsu Fall 2018 Alumna | Finer Women | Passion for Bench to Bedside & Public Health | Founder of @babescientists | Follow my Humorous Scientific Journey,Fall 2018 Alumna | Finer Women | Passion for Bench to Bedside & Public Health | Founder of | Follow my Humorous Scientific Journey,Fayetteville; NC,724096087,Fayetteville,NC,Fayetteville; NC
999997,41436303,2019-01-04 23:03:12,New York; USA,you cannot kill me in a way that matter | 180 cm of I SAID WHAT I SAID | MD in the making | Fighting evil by moonlight|,you cannot kill me in a way that matter | 180 cm of I SAID WHAT I SAID | MD in the making | Fighting evil by moonlight|,New York; USA,41436303,New York,US,New York; US


# Retrieval from USPS zipcode

In [841]:
zipcode_file = r'https://postalpro.usps.com/mnt/glusterfs/2023-06/ZIP_Locale_Detail.xls'   # https://postalpro.usps.com/ZIP_Locale_Detail

zipcode_df = pd.read_excel(zipcode_file, sheet_name='ZIP_DETAIL')
zipcode_df['USPS_city'] = zipcode_df['PHYSICAL CITY'].str.strip().str.title()
zipcode_df['USPS_state'] = zipcode_df['PHYSICAL STATE'].str.strip().str.upper()
zipcode_df['USPS_city_state'] = zipcode_df['USPS_city'] + '; ' + zipcode_df['USPS_state']

# zipcode_df['USPS_city_state'] = zipcode_df['city_state']
zipcode_df = zipcode_df.groupby(['USPS_city', 'USPS_state', 'USPS_city_state'], as_index=False)['DISTRICT NAME'].count()
zipcode_df

,USPS_city,USPS_state,USPS_city_state,DISTRICT NAME
0,Aaronsburg,PA,Aaronsburg; PA,1
1,Abbeville,AL,Abbeville; AL,3
2,Abbeville,GA,Abbeville; GA,1
3,Abbeville,LA,Abbeville; LA,2
4,Abbeville,MS,Abbeville; MS,1
...,...,...,...,...
27049,Zuni,NM,Zuni; NM,1
27050,Zuni,VA,Zuni; VA,1
27051,Zurich,MT,Zurich; MT,1
27052,Zwingle,IA,Zwingle; IA,1


In [842]:
merged_user_df = raw_user_df.merge(zipcode_df['USPS_city_state'], how='left', left_on='user_city_state', right_on=r'USPS_city_state')
matched_idx = ~merged_user_df['USPS_city_state'].isna()
matched_user_df = merged_user_df[matched_idx] 
matched_user_df

,userID,date,userLocation,userDescription,description,location,id,user_city,user_state,user_city_state,USPS_city_state
2,1596576888,2017-02-02 11:33:39,San Jacinto; CA,S'il vous plat; don't you hate,S'il vous plat; don't you hate,San Jacinto; CA,1596576888,San Jacinto,CA,San Jacinto; CA,San Jacinto; CA
8,30004656,2017-02-02 11:33:39,Springfield; Illinois,NAI True Commercial Real Estate Services. A worldwide network of over 6;700 professionals & 375 offices. Committed to Central Illinois. Connected to the World.,NAI True Commercial Real Estate Services. A worldwide network of over 6;700 professionals & 375 offices. Committed to Central Illinois. Connected to the World.,Springfield; Illinois,30004656,Springfield,IL,Springfield; IL,Springfield; IL
11,235793030,2017-02-02 11:33:39,Philadelphia; PA,This is my part nobody else speak. #widener18,This is my part nobody else speak. #widener18,Philadelphia; PA,235793030,Philadelphia,PA,Philadelphia; PA,Philadelphia; PA
17,997306855,2017-02-02 11:33:40,Hartford; Al,I am his.,I am his.,Hartford; Al,997306855,Hartford,AL,Hartford; AL,Hartford; AL
22,783396742473650177,2017-02-02 11:33:39,Washington; DC,The National LGBTQ Task Force Action Fund conducts grassroots organizing and lobbying on legislation and ballot initiatives to achieve justice for #LGBTQ people,The National LGBTQ Task Force Action Fund conducts grassroots organizing and lobbying on legislation and ballot initiatives to achieve justice for #LGBTQ people,Washington; DC,783396742473650177,Washington,DC,Washington; DC,Washington; DC
...,...,...,...,...,...,...,...,...,...,...,...
999981,1051977810,2019-01-04 23:03:12,Visalia; CA,#BeautifulMind,#BeautifulMind,Visalia; CA,1051977810,Visalia,CA,Visalia; CA,Visalia; CA
999984,838822579414663168,2019-01-04 23:03:12,Bayport; NY,Music; Beaches; Red Rock; gem stones; animals and most of all..;my family.,Music; Beaches; Red Rock; gem stones; animals and most of all..;my family.,Bayport; NY,838822579414663168,Bayport,NY,Bayport; NY,Bayport; NY
999988,408899228,2019-01-04 23:03:12,Tallahassee; FL,@UF grad and proud @UFJSchool alum. Weekend Sports Anchor/Sports Reporter @abc27. Biggest Jacksonville Jaguars fan you will ever meet.,grad and proud alum. Weekend Sports Anchor/Sports Reporter . Biggest Jacksonville Jaguars fan you will ever meet.,Tallahassee; FL,408899228,Tallahassee,FL,Tallahassee; FL,Tallahassee; FL
999996,724096087,2019-01-04 23:03:12,Fayetteville; NC,@uncfsu Fall 2018 Alumna | Finer Women | Passion for Bench to Bedside & Public Health | Founder of @babescientists | Follow my Humorous Scientific Journey,Fall 2018 Alumna | Finer Women | Passion for Bench to Bedside & Public Health | Founder of | Follow my Humorous Scientific Journey,Fayetteville; NC,724096087,Fayetteville,NC,Fayetteville; NC,Fayetteville; NC


In [843]:
unmatched_user_df = merged_user_df[~matched_idx] 
unmatched_user_df

,userID,date,userLocation,userDescription,description,location,id,user_city,user_state,user_city_state,USPS_city_state
0,1064238776,2017-02-02 11:33:38,,IXVIIIXVII fly high babygirl,IXVIIIXVII fly high babygirl,,1064238776,NaN,,NaN,NaN
1,479920728,2017-02-02 11:33:39,where im meant to be,,,where im meant to be,479920728,NaN,WHERE IM MEANT TO BE,NaN,NaN
3,748510143487279105,2017-02-02 11:33:39,67th Langley,,,67th Langley,748510143487279105,NaN,67TH LANGLEY,NaN,NaN
4,17754800,2017-02-02 11:33:39,,pioneer sound // delta & dawn,pioneer sound // delta & dawn,,17754800,NaN,,NaN,NaN
5,322642659,2017-02-02 11:33:39,Maryland,Social Justice minded United Methodist Deacon; spouse of an #UMC Elder; mom of an exceptional aspie; lifetime member of the Lyme disease community,Social Justice minded United Methodist Deacon; spouse of an #UMC Elder; mom of an exceptional aspie; lifetime member of the Lyme disease community,Maryland,322642659,NaN,MD,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
999992,2772131425,2019-01-04 23:03:12,HopkinsDecaturMissouri,Culver Stockton College MBBInstagram:kevins_hoopdreams; Snapchat:kevin_shifty,Culver Stockton College MBBInstagram:kevins_hoopdreams; Snapchat:kevin_shifty,HopkinsDecaturMissouri,2772131425,NaN,HOPKINSDECATURMISSOURI,NaN,NaN
999993,618428269,2019-01-04 23:03:12,Michigan; USA,University of Michigan '21 sc// averyannh,University of Michigan '21 sc// averyannh,Michigan; USA,618428269,Michigan,US,Michigan; US,NaN
999994,242075925,2019-01-04 23:03:12,,alani,alani,,242075925,NaN,,NaN,NaN
999995,3517448480,2019-01-04 23:03:12,Florida; USA,Independent.Supports equality and gun control. Racism makes me sick; we are bigger and better than this. 1 tribe yall Trump is an epic scumbag Idiot.,Independent.Supports equality and gun control. Racism makes me sick; we are bigger and better than this. 1 tribe yall Trump is an epic scumbag Idiot.,Florida; USA,3517448480,Florida,US,Florida; US,NaN


In [479]:
unmatched_user_df.sample(10)[['location', 'user_city', 'user_state', 'user_city_state']]

,location,user_city,user_state,user_city_state
805272,w/ my baby.,NaN,W/ MY BABY.,NaN
769518,Austin,NaN,AUSTIN,NaN
11665,California; USA,California,USA,California; USA
137495,São Sebastião do Anta,NaN,SÃO SEBASTIÃO DO ANTA,NaN
565688,Everywhere,NaN,EVERYWHERE,NaN
724481,,NaN,,NaN
506816,,NaN,,NaN
158972,Looziana💥🐊🐎,NaN,LOOZIANA💥🐊🐎,NaN
593976,,NaN,,NaN
876516,510,NaN,510,NaN


In [844]:
print("Total user self-reported location, unmatche location, matched location:")
len(raw_user_df['userLocation'].unique()), len(unmatched_user_df['userLocation'].unique()), len(matched_user_df['userLocation'].unique())

Total user self-reported location, unmatche location, matched location:


(144795, 126531, 18264)

In [845]:
print("Total tweets, unmatche tweet rate, matched tweet rate:")
total_tweet_cnt = len(raw_user_df['userLocation']) 
total_tweet_cnt, len(unmatched_user_df['userLocation']) / total_tweet_cnt, len(matched_user_df['userLocation'] )  / total_tweet_cnt

Total tweets, unmatche tweet rate, matched tweet rate:


(999999, 0.6437046437046438, 0.3562953562953563)

## Check the state level

In [846]:
unmatched_user_df.query('user_state == "US" ')#.sample(10)[['location', 'user_city', 'user_state', 'user_city_state']]

,userID,date,userLocation,userDescription,description,location,id,user_city,user_state,user_city_state,USPS_city_state
7,3302332229,2017-02-02 11:33:39,Kentucky; USA,Model for Waffle House,Model for Waffle House,Kentucky; USA,3302332229,Kentucky,US,Kentucky; US,NaN
40,419989279,2017-02-02 11:33:40,United States,,,United States,419989279,NaN,US,NaN,NaN
66,2228641,2017-02-02 11:33:42,New York; USA,A feminist; researcher; and entrepreneur. A @columbia Alumnus. My tweets are in English and Persian. RT#endorsement. http://www.docademia.com,A feminist; researcher; and entrepreneur. A Alumnus. My tweets are in English and Persian. RT#endorsement.,New York; USA,2228641,New York,US,New York; US,NaN
73,799371881375744001,2017-02-02 11:33:42,Kentucky; USA,Essayist wrestling with a memoir. Tiny blue dot in a red state @heraldleader. Potato chips and kindness appreciated.,Essayist wrestling with a memoir. Tiny blue dot in a red state . Potato chips and kindness appreciated.,Kentucky; USA,799371881375744001,Kentucky,US,Kentucky; US,NaN
92,2598217694,2017-02-02 11:33:43,usa,Host of @theweeklytop_40 & @dadoodirtyshow (Saturdays) Program Dir at Alldigitalradio! My r my Boyfriend/Weed/ & Politics. Follow IG: justdoodirty,Host of & (Saturdays) Program Dir at Alldigitalradio! My r my Boyfriend/Weed/ & Politics. Follow IG: justdoodirty,usa,2598217694,NaN,US,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
999943,861038103384993792,2019-01-04 23:03:10,North Carolina; USA,19 | streets said I was funny and you should follow me,19 | streets said I was funny and you should follow me,North Carolina; USA,861038103384993792,North Carolina,US,North Carolina; US,NaN
999949,1022263433432104960,2019-01-04 23:03:09,Missouri; USA,oklahoma As 16u premier-madden #00 RHP/ corners MO southland high 4.0 GPA nsrjenna.gibson21@gmail.com,oklahoma As 16u premier-madden #00 RHP/ corners MO southland high 4.0 GPA nsrjenna.gibson21.com,Missouri; USA,1022263433432104960,Missouri,US,Missouri; US,NaN
999993,618428269,2019-01-04 23:03:12,Michigan; USA,University of Michigan '21 sc// averyannh,University of Michigan '21 sc// averyannh,Michigan; USA,618428269,Michigan,US,Michigan; US,NaN
999995,3517448480,2019-01-04 23:03:12,Florida; USA,Independent.Supports equality and gun control. Racism makes me sick; we are bigger and better than this. 1 tribe yall Trump is an epic scumbag Idiot.,Independent.Supports equality and gun control. Racism makes me sick; we are bigger and better than this. 1 tribe yall Trump is an epic scumbag Idiot.,Florida; USA,3517448480,Florida,US,Florida; US,NaN


In [847]:
state_abbr_df.head()

,State,Abbreviation,state_US
0,ALABAMA,AL,Alabama; US
1,ALASKA,AK,Alaska; US
2,AMERICAN SAMOA,AS,American Samoa; US
3,ARIZONA,AZ,Arizona; US
4,ARKANSAS,AR,Arkansas; US


In [848]:
merged_state_user_df = unmatched_user_df.merge(state_abbr_df['state_US'], how = 'left',  left_on='user_city_state', right_on=r'state_US')
state_matched_idx = ~merged_state_user_df['state_US'].isna()
state_matched_user_df = merged_state_user_df[state_matched_idx] 
state_matched_user_df


,userID,date,userLocation,userDescription,description,location,id,user_city,user_state,user_city_state,USPS_city_state,state_US
6,3302332229,2017-02-02 11:33:39,Kentucky; USA,Model for Waffle House,Model for Waffle House,Kentucky; USA,3302332229,Kentucky,US,Kentucky; US,NaN,Kentucky; US
50,2228641,2017-02-02 11:33:42,New York; USA,A feminist; researcher; and entrepreneur. A @columbia Alumnus. My tweets are in English and Persian. RT#endorsement. http://www.docademia.com,A feminist; researcher; and entrepreneur. A Alumnus. My tweets are in English and Persian. RT#endorsement.,New York; USA,2228641,New York,US,New York; US,NaN,New York; US
55,799371881375744001,2017-02-02 11:33:42,Kentucky; USA,Essayist wrestling with a memoir. Tiny blue dot in a red state @heraldleader. Potato chips and kindness appreciated.,Essayist wrestling with a memoir. Tiny blue dot in a red state . Potato chips and kindness appreciated.,Kentucky; USA,799371881375744001,Kentucky,US,Kentucky; US,NaN,Kentucky; US
74,238776270,2017-02-02 11:33:44,Arizona; USA,La ilahe illa Allah. Nokta. For English tweets follow: @19org Read Quran: a Reformist Translation - Manifesto for Islamic Reform.o,La ilahe illa Allah. Nokta. For English tweets follow: Read Quran: a Reformist Translation - Manifesto for Islamic Reform.o,Arizona; USA,238776270,Arizona,US,Arizona; US,NaN,Arizona; US
94,216188796,2017-02-02 11:33:45,Maryland; USA,GO TERPS! Ocean city Elvis & Motown BS Graphic Design I love Music & dancing,GO TERPS! Ocean city Elvis & Motown BS Graphic Design I love Music & dancing,Maryland; USA,216188796,Maryland,US,Maryland; US,NaN,Maryland; US
...,...,...,...,...,...,...,...,...,...,...,...,...
643672,861038103384993792,2019-01-04 23:03:10,North Carolina; USA,19 | streets said I was funny and you should follow me,19 | streets said I was funny and you should follow me,North Carolina; USA,861038103384993792,North Carolina,US,North Carolina; US,NaN,North Carolina; US
643674,1022263433432104960,2019-01-04 23:03:09,Missouri; USA,oklahoma As 16u premier-madden #00 RHP/ corners MO southland high 4.0 GPA nsrjenna.gibson21@gmail.com,oklahoma As 16u premier-madden #00 RHP/ corners MO southland high 4.0 GPA nsrjenna.gibson21.com,Missouri; USA,1022263433432104960,Missouri,US,Missouri; US,NaN,Missouri; US
643700,618428269,2019-01-04 23:03:12,Michigan; USA,University of Michigan '21 sc// averyannh,University of Michigan '21 sc// averyannh,Michigan; USA,618428269,Michigan,US,Michigan; US,NaN,Michigan; US
643702,3517448480,2019-01-04 23:03:12,Florida; USA,Independent.Supports equality and gun control. Racism makes me sick; we are bigger and better than this. 1 tribe yall Trump is an epic scumbag Idiot.,Independent.Supports equality and gun control. Racism makes me sick; we are bigger and better than this. 1 tribe yall Trump is an epic scumbag Idiot.,Florida; USA,3517448480,Florida,US,Florida; US,NaN,Florida; US


In [849]:
state_unmatched_user_df = merged_state_user_df[~state_matched_idx].fillna("")
state_unmatched_user_df

,userID,date,userLocation,userDescription,description,location,id,user_city,user_state,user_city_state,USPS_city_state,state_US
0,1064238776,2017-02-02 11:33:38,,IXVIIIXVII fly high babygirl,IXVIIIXVII fly high babygirl,,1064238776,,,,,
1,479920728,2017-02-02 11:33:39,where im meant to be,,,where im meant to be,479920728,,WHERE IM MEANT TO BE,,,
2,748510143487279105,2017-02-02 11:33:39,67th Langley,,,67th Langley,748510143487279105,,67TH LANGLEY,,,
3,17754800,2017-02-02 11:33:39,,pioneer sound // delta & dawn,pioneer sound // delta & dawn,,17754800,,,,,
4,322642659,2017-02-02 11:33:39,Maryland,Social Justice minded United Methodist Deacon; spouse of an #UMC Elder; mom of an exceptional aspie; lifetime member of the Lyme disease community,Social Justice minded United Methodist Deacon; spouse of an #UMC Elder; mom of an exceptional aspie; lifetime member of the Lyme disease community,Maryland,322642659,,MD,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
643696,43212382,2019-01-04 23:03:12,International,Public Relations Professional. 26. #AKA1908,Public Relations Professional. 26. #AKA1908,International,43212382,,INTERNATIONAL,,,
643697,4627173088,2019-01-04 23:03:12,,Hunter Bailey,Hunter Bailey,,4627173088,,,,,
643698,1042528723692601345,2019-01-04 23:03:12,,bout my guap not you,bout my guap not you,,1042528723692601345,,,,,
643699,2772131425,2019-01-04 23:03:12,HopkinsDecaturMissouri,Culver Stockton College MBBInstagram:kevins_hoopdreams; Snapchat:kevin_shifty,Culver Stockton College MBBInstagram:kevins_hoopdreams; Snapchat:kevin_shifty,HopkinsDecaturMissouri,2772131425,,HOPKINSDECATURMISSOURI,,,


In [906]:
country_df = state_unmatched_user_df.query('user_state == "US"  ')#sample(10) # and user_city == ""
print("country user rate:")
len(country_df['id'].unique()) / total_user_cnt

country user rate:


0.017495539273376107

In [850]:

state_unmatched_user_df.query('user_state == ""  and user_city == "" ')#.sample(10) # and user_city == ""

,userID,date,userLocation,userDescription,description,location,id,user_city,user_state,user_city_state,USPS_city_state,state_US
0,1064238776,2017-02-02 11:33:38,,IXVIIIXVII fly high babygirl,IXVIIIXVII fly high babygirl,,1064238776,,,,,
3,17754800,2017-02-02 11:33:39,,pioneer sound // delta & dawn,pioneer sound // delta & dawn,,17754800,,,,,
10,1388583182,2017-02-02 11:33:40,,stop breathing bitch,stop breathing bitch,,1388583182,,,,,
13,2876978609,2017-02-02 11:33:40,,Curly Q. Gran,Curly Q. Gran,,2876978609,,,,,
19,2783371728,2017-02-02 11:33:40,,ig // snap- @alliece.34,ig // snap- .34,,2783371728,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
643693,752368134,2019-01-04 23:03:12,,probably crying over some homework or something,probably crying over some homework or something,,752368134,,,,,
643694,3077485109,2019-01-04 23:03:12,,,,,3077485109,,,,,
643697,4627173088,2019-01-04 23:03:12,,Hunter Bailey,Hunter Bailey,,4627173088,,,,,
643698,1042528723692601345,2019-01-04 23:03:12,,bout my guap not you,bout my guap not you,,1042528723692601345,,,,,


# Census data matching

In [851]:
zipcode_df.merge(place_df, how='left', left_on='USPS_city_state', right_on='place_city_state')#['place_city_state'].isna().sum()

,USPS_city,USPS_state,USPS_city_state,DISTRICT NAME,place_city,place_state,place_city_state,NAME
0,Aaronsburg,PA,Aaronsburg; PA,1,Aaronsburg,PA,Aaronsburg; PA,2.0
1,Abbeville,AL,Abbeville; AL,3,Abbeville,AL,Abbeville; AL,1.0
2,Abbeville,GA,Abbeville; GA,1,Abbeville,GA,Abbeville; GA,1.0
3,Abbeville,LA,Abbeville; LA,2,Abbeville,LA,Abbeville; LA,1.0
4,Abbeville,MS,Abbeville; MS,1,Abbeville,MS,Abbeville; MS,1.0
...,...,...,...,...,...,...,...,...
27049,Zuni,NM,Zuni; NM,1,NaN,NaN,NaN,NaN
27050,Zuni,VA,Zuni; VA,1,NaN,NaN,NaN,NaN
27051,Zurich,MT,Zurich; MT,1,Zurich,MT,Zurich; MT,1.0
27052,Zwingle,IA,Zwingle; IA,1,Zwingle,IA,Zwingle; IA,1.0


In [852]:
place_TSV = r'https://www2.census.gov/geo/docs/maps-data/data/gazetteer/2022_Gazetteer/2022_Gaz_place_national.zip'
place_df = pd.read_csv(place_TSV, delimiter='\t')
place_df['place_city'] = place_df['NAME']
place_df['place_city'] = place_df['place_city'].str.split(" ").apply(lambda x: ' '.join(x[:-1]))
place_df['place_city'] = place_df['place_city'].str.strip().str.title()
place_df['place_state'] = place_df['USPS'].str.strip().str.upper()
place_df['place_city_state'] = place_df['place_city'] + '; ' + place_df['place_state']
# zipcode_df['USPS_city_state'] = zipcode_df['city_state']
place_df = place_df.groupby(['place_city', 'place_state', 'place_city_state'], as_index=False)['NAME'].count()
place_df

place_df

,place_city,place_state,place_city_state,NAME
0,,NJ,; NJ,1
1,Aaronsburg,PA,Aaronsburg; PA,2
2,Abanda,AL,Abanda; AL,1
3,Abbeville,AL,Abbeville; AL,1
4,Abbeville,GA,Abbeville; GA,1
...,...,...,...,...
31969,Zuni Pueblo,NM,Zuni Pueblo; NM,1
31970,Zurich,KS,Zurich; KS,1
31971,Zurich,MT,Zurich; MT,1
31972,Zwingle,IA,Zwingle; IA,1


In [500]:
# merged_user_df

In [853]:
merged_place_user_df = raw_user_df.merge(place_df['place_city_state'], how='left', left_on='user_city_state', right_on=r'place_city_state')
matched_place_idx = ~merged_place_user_df['place_city_state'].isna()
matched_place_user_df = merged_place_user_df[matched_place_idx] 
matched_place_user_df

,userID,date,userLocation,userDescription,description,location,id,user_city,user_state,user_city_state,place_city_state
2,1596576888,2017-02-02 11:33:39,San Jacinto; CA,S'il vous plat; don't you hate,S'il vous plat; don't you hate,San Jacinto; CA,1596576888,San Jacinto,CA,San Jacinto; CA,San Jacinto; CA
8,30004656,2017-02-02 11:33:39,Springfield; Illinois,NAI True Commercial Real Estate Services. A worldwide network of over 6;700 professionals & 375 offices. Committed to Central Illinois. Connected to the World.,NAI True Commercial Real Estate Services. A worldwide network of over 6;700 professionals & 375 offices. Committed to Central Illinois. Connected to the World.,Springfield; Illinois,30004656,Springfield,IL,Springfield; IL,Springfield; IL
11,235793030,2017-02-02 11:33:39,Philadelphia; PA,This is my part nobody else speak. #widener18,This is my part nobody else speak. #widener18,Philadelphia; PA,235793030,Philadelphia,PA,Philadelphia; PA,Philadelphia; PA
17,997306855,2017-02-02 11:33:40,Hartford; Al,I am his.,I am his.,Hartford; Al,997306855,Hartford,AL,Hartford; AL,Hartford; AL
22,783396742473650177,2017-02-02 11:33:39,Washington; DC,The National LGBTQ Task Force Action Fund conducts grassroots organizing and lobbying on legislation and ballot initiatives to achieve justice for #LGBTQ people,The National LGBTQ Task Force Action Fund conducts grassroots organizing and lobbying on legislation and ballot initiatives to achieve justice for #LGBTQ people,Washington; DC,783396742473650177,Washington,DC,Washington; DC,Washington; DC
...,...,...,...,...,...,...,...,...,...,...,...
999981,1051977810,2019-01-04 23:03:12,Visalia; CA,#BeautifulMind,#BeautifulMind,Visalia; CA,1051977810,Visalia,CA,Visalia; CA,Visalia; CA
999984,838822579414663168,2019-01-04 23:03:12,Bayport; NY,Music; Beaches; Red Rock; gem stones; animals and most of all..;my family.,Music; Beaches; Red Rock; gem stones; animals and most of all..;my family.,Bayport; NY,838822579414663168,Bayport,NY,Bayport; NY,Bayport; NY
999988,408899228,2019-01-04 23:03:12,Tallahassee; FL,@UF grad and proud @UFJSchool alum. Weekend Sports Anchor/Sports Reporter @abc27. Biggest Jacksonville Jaguars fan you will ever meet.,grad and proud alum. Weekend Sports Anchor/Sports Reporter . Biggest Jacksonville Jaguars fan you will ever meet.,Tallahassee; FL,408899228,Tallahassee,FL,Tallahassee; FL,Tallahassee; FL
999996,724096087,2019-01-04 23:03:12,Fayetteville; NC,@uncfsu Fall 2018 Alumna | Finer Women | Passion for Bench to Bedside & Public Health | Founder of @babescientists | Follow my Humorous Scientific Journey,Fall 2018 Alumna | Finer Women | Passion for Bench to Bedside & Public Health | Founder of | Follow my Humorous Scientific Journey,Fayetteville; NC,724096087,Fayetteville,NC,Fayetteville; NC,Fayetteville; NC


In [854]:
unmatched_place_user_df = merged_place_user_df[~matched_place_idx] 
unmatched_place_user_df

,userID,date,userLocation,userDescription,description,location,id,user_city,user_state,user_city_state,place_city_state
0,1064238776,2017-02-02 11:33:38,,IXVIIIXVII fly high babygirl,IXVIIIXVII fly high babygirl,,1064238776,NaN,,NaN,NaN
1,479920728,2017-02-02 11:33:39,where im meant to be,,,where im meant to be,479920728,NaN,WHERE IM MEANT TO BE,NaN,NaN
3,748510143487279105,2017-02-02 11:33:39,67th Langley,,,67th Langley,748510143487279105,NaN,67TH LANGLEY,NaN,NaN
4,17754800,2017-02-02 11:33:39,,pioneer sound // delta & dawn,pioneer sound // delta & dawn,,17754800,NaN,,NaN,NaN
5,322642659,2017-02-02 11:33:39,Maryland,Social Justice minded United Methodist Deacon; spouse of an #UMC Elder; mom of an exceptional aspie; lifetime member of the Lyme disease community,Social Justice minded United Methodist Deacon; spouse of an #UMC Elder; mom of an exceptional aspie; lifetime member of the Lyme disease community,Maryland,322642659,NaN,MD,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
999992,2772131425,2019-01-04 23:03:12,HopkinsDecaturMissouri,Culver Stockton College MBBInstagram:kevins_hoopdreams; Snapchat:kevin_shifty,Culver Stockton College MBBInstagram:kevins_hoopdreams; Snapchat:kevin_shifty,HopkinsDecaturMissouri,2772131425,NaN,HOPKINSDECATURMISSOURI,NaN,NaN
999993,618428269,2019-01-04 23:03:12,Michigan; USA,University of Michigan '21 sc// averyannh,University of Michigan '21 sc// averyannh,Michigan; USA,618428269,Michigan,US,Michigan; US,NaN
999994,242075925,2019-01-04 23:03:12,,alani,alani,,242075925,NaN,,NaN,NaN
999995,3517448480,2019-01-04 23:03:12,Florida; USA,Independent.Supports equality and gun control. Racism makes me sick; we are bigger and better than this. 1 tribe yall Trump is an epic scumbag Idiot.,Independent.Supports equality and gun control. Racism makes me sick; we are bigger and better than this. 1 tribe yall Trump is an epic scumbag Idiot.,Florida; USA,3517448480,Florida,US,Florida; US,NaN


In [855]:
print("Total user self-reported location, unmatche location, matched location:")
len(raw_user_df['userLocation'].unique()), len(unmatched_place_user_df['userLocation'].unique()), len(matched_place_user_df['userLocation'].unique())

Total user self-reported location, unmatche location, matched location:


(144795, 126198, 18597)

In [876]:
print("Total tweets, unmatche tweet rate, matched tweet rate:")
total_tweet_cnt = len(raw_user_df['userLocation']) 
total_tweet_cnt / total_tweet_cnt, len(unmatched_place_user_df['userLocation']) / total_tweet_cnt, len(matched_place_user_df['userLocation'] )  / total_tweet_cnt

Total tweets, unmatche tweet rate, matched tweet rate:


(1.0, 0.6545576545576546, 0.3454423454423454)

In [873]:
print("Total user, unmatche user rate, matched user rate:")
total_user_cnt = len(raw_user_df['id'].unique()) 
total_user_cnt, len(unmatched_place_user_df['id'].unique()) / total_user_cnt, len(matched_place_user_df['id'].unique() )  / total_user_cnt

Total user, unmatche user rate, matched user rate:


(526820, 0.6613606165293648, 0.35773129342090276)

In [875]:
print("Total user, unmatche user, matched user:")
total_user_cnt = len(raw_user_df['id'].unique()) 
total_user_cnt, len(unmatched_place_user_df['id'].unique())  , len(matched_place_user_df['id'].unique() ) 

Total user, unmatche user, matched user:


(526820, 348418, 188460)

In [883]:
# unmatched_place_user_df.query("user_state == 'US' ")#.sample(20)#[['userLocation', 'user_city_state']]
# unmatched_place_user_df.query("user_state == 'USA' ").sample(20)[['userLocation', 'user_city']]
# unmatched_place_user_df.sample(20)[['userLocation', 'user_city_state']]
empty_location_df = raw_user_df.query("location == '' ")
print("Empty profile location user coucnt:")
len(empty_location_df['id'].unique()) / total_user_cnt

Empty profile location user coucnt:


0.18672601647621578

In [858]:
unmatched_place_user_df['user_state'].value_counts()
# unmatched_place_user_df.query("user_state == 'USA' ") 

                                  162617
US                                79051 
NY                                16935 
ONTARIO                           8422  
TX                                7085  
                                  ...   
CHICAGO / TORONTO / EAST COAST    1     
BOWLING GREEN. UNFORTUNATELY      1     
UT/CA                             1     
GRAA                              1     
HOPKINSDECATURMISSOURI            1     
Name: user_state, Length: 87795, dtype: int64

## clean the state name for the unmatched state

In [859]:
unmatched_place_user_df['user_state'].value_counts()
# raw_user_df['user_state'].value_counts()

                                  162617
US                                79051 
NY                                16935 
ONTARIO                           8422  
TX                                7085  
                                  ...   
CHICAGO / TORONTO / EAST COAST    1     
BOWLING GREEN. UNFORTUNATELY      1     
UT/CA                             1     
GRAA                              1     
HOPKINSDECATURMISSOURI            1     
Name: user_state, Length: 87795, dtype: int64

In [437]:
# unmatched_place_user_df['user_state'] = unmatched_place_user_df['user_state'].str.title()
# # unmatched_place_user_df.merge(state_abbr_df, left_on='user_state', right_on='State')

In [885]:
# state_abbr_df
unmatched_place_user_df

,userID,date,userLocation,userDescription,description,location,id,user_city,user_state,user_city_state,place_city_state
0,1064238776,2017-02-02 11:33:38,,IXVIIIXVII fly high babygirl,IXVIIIXVII fly high babygirl,,1064238776,NaN,,NaN,NaN
1,479920728,2017-02-02 11:33:39,where im meant to be,,,where im meant to be,479920728,NaN,WHERE IM MEANT TO BE,NaN,NaN
3,748510143487279105,2017-02-02 11:33:39,67th Langley,,,67th Langley,748510143487279105,NaN,67TH LANGLEY,NaN,NaN
4,17754800,2017-02-02 11:33:39,,pioneer sound // delta & dawn,pioneer sound // delta & dawn,,17754800,NaN,,NaN,NaN
5,322642659,2017-02-02 11:33:39,Maryland,Social Justice minded United Methodist Deacon; spouse of an #UMC Elder; mom of an exceptional aspie; lifetime member of the Lyme disease community,Social Justice minded United Methodist Deacon; spouse of an #UMC Elder; mom of an exceptional aspie; lifetime member of the Lyme disease community,Maryland,322642659,NaN,MD,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
999992,2772131425,2019-01-04 23:03:12,HopkinsDecaturMissouri,Culver Stockton College MBBInstagram:kevins_hoopdreams; Snapchat:kevin_shifty,Culver Stockton College MBBInstagram:kevins_hoopdreams; Snapchat:kevin_shifty,HopkinsDecaturMissouri,2772131425,NaN,HOPKINSDECATURMISSOURI,NaN,NaN
999993,618428269,2019-01-04 23:03:12,Michigan; USA,University of Michigan '21 sc// averyannh,University of Michigan '21 sc// averyannh,Michigan; USA,618428269,Michigan,US,Michigan; US,NaN
999994,242075925,2019-01-04 23:03:12,,alani,alani,,242075925,NaN,,NaN,NaN
999995,3517448480,2019-01-04 23:03:12,Florida; USA,Independent.Supports equality and gun control. Racism makes me sick; we are bigger and better than this. 1 tribe yall Trump is an epic scumbag Idiot.,Independent.Supports equality and gun control. Racism makes me sick; we are bigger and better than this. 1 tribe yall Trump is an epic scumbag Idiot.,Florida; USA,3517448480,Florida,US,Florida; US,NaN


In [887]:
merged_state_user_df = unmatched_place_user_df.merge(state_abbr_df['state_US'], how = 'left',  left_on='user_city_state', right_on=r'state_US')
state_matched_idx = ~merged_state_user_df['state_US'].isna()
state_matched_user_df = merged_state_user_df[state_matched_idx] 

print("State user rate:")
len(state_matched_user_df['id'].unique()) / total_user_cnt


State user rate:


0.06125431836300824

In [888]:
state_unmatched_user_df = merged_state_user_df[~state_matched_idx].fillna("")
state_unmatched_user_df

,userID,date,userLocation,userDescription,description,location,id,user_city,user_state,user_city_state,place_city_state,state_US
0,1064238776,2017-02-02 11:33:38,,IXVIIIXVII fly high babygirl,IXVIIIXVII fly high babygirl,,1064238776,,,,,
1,479920728,2017-02-02 11:33:39,where im meant to be,,,where im meant to be,479920728,,WHERE IM MEANT TO BE,,,
2,748510143487279105,2017-02-02 11:33:39,67th Langley,,,67th Langley,748510143487279105,,67TH LANGLEY,,,
3,17754800,2017-02-02 11:33:39,,pioneer sound // delta & dawn,pioneer sound // delta & dawn,,17754800,,,,,
4,322642659,2017-02-02 11:33:39,Maryland,Social Justice minded United Methodist Deacon; spouse of an #UMC Elder; mom of an exceptional aspie; lifetime member of the Lyme disease community,Social Justice minded United Methodist Deacon; spouse of an #UMC Elder; mom of an exceptional aspie; lifetime member of the Lyme disease community,Maryland,322642659,,MD,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
654549,43212382,2019-01-04 23:03:12,International,Public Relations Professional. 26. #AKA1908,Public Relations Professional. 26. #AKA1908,International,43212382,,INTERNATIONAL,,,
654550,4627173088,2019-01-04 23:03:12,,Hunter Bailey,Hunter Bailey,,4627173088,,,,,
654551,1042528723692601345,2019-01-04 23:03:12,,bout my guap not you,bout my guap not you,,1042528723692601345,,,,,
654552,2772131425,2019-01-04 23:03:12,HopkinsDecaturMissouri,Culver Stockton College MBBInstagram:kevins_hoopdreams; Snapchat:kevin_shifty,Culver Stockton College MBBInstagram:kevins_hoopdreams; Snapchat:kevin_shifty,HopkinsDecaturMissouri,2772131425,,HOPKINSDECATURMISSOURI,,,


In [889]:
state_unmatched_user_df['location'] = state_unmatched_user_df['location'].str.strip()
empth_loc_idx = state_unmatched_user_df['location'] == ""
empth_loc_df = state_unmatched_user_df[empth_loc_idx]
not_empty_unmatched_user_df = state_unmatched_user_df[~empth_loc_idx]
not_empty_unmatched_user_df

,userID,date,userLocation,userDescription,description,location,id,user_city,user_state,user_city_state,place_city_state,state_US
1,479920728,2017-02-02 11:33:39,where im meant to be,,,where im meant to be,479920728,,WHERE IM MEANT TO BE,,,
2,748510143487279105,2017-02-02 11:33:39,67th Langley,,,67th Langley,748510143487279105,,67TH LANGLEY,,,
4,322642659,2017-02-02 11:33:39,Maryland,Social Justice minded United Methodist Deacon; spouse of an #UMC Elder; mom of an exceptional aspie; lifetime member of the Lyme disease community,Social Justice minded United Methodist Deacon; spouse of an #UMC Elder; mom of an exceptional aspie; lifetime member of the Lyme disease community,Maryland,322642659,,MD,,,
5,34476701,2017-02-02 11:33:39,Illinois,MERICA ! under construction - cars are life,MERICA ! under construction - cars are life,Illinois,34476701,,IL,,,
7,107890250,2017-02-02 11:33:39,LosAngeles-Heidelberg-Caesarea,A good-for-nothing crusty old bugger Editor and biblical scholar with PhD became a buskerhttp://youtu.be/QYxNiKiMwCI,A good-for-nothing crusty old bugger Editor and biblical scholar with PhD became a busker,LosAngeles-Heidelberg-Caesarea,107890250,,LOSANGELES-HEIDELBERG-CAESAREA,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
654542,114966176,2019-01-04 23:03:11,CT/NJ Bay Area,Founder; @fourth_nine | Cover Four Podcast on @iTunes | @UConn alum | Previously: @NinersNation; @BleacherReport & @NBCSAuthentic | https://apple.co/2soCa5l,Founder; | Cover Four Podcast on | alum | Previously: ; & |,CT/NJ Bay Area,114966176,,CT/NJ BAY AREA,,,
654543,2375578866,2019-01-04 23:03:11,America,,,America,2375578866,,AMERICA,,,
654548,134105717,2019-01-04 23:03:12,tuscaloosa,my son; my moon ; and all of my stars. 03-13-18,my son; my moon ; and all of my stars. 03-13-18,tuscaloosa,134105717,,TUSCALOOSA,,,
654549,43212382,2019-01-04 23:03:12,International,Public Relations Professional. 26. #AKA1908,Public Relations Professional. 26. #AKA1908,International,43212382,,INTERNATIONAL,,,


In [732]:
# # state_abbr_df['Abbreviation'].to_list()
# other_state_idx

In [900]:
# unlocalized_df
# other_state_df
not_empty_unmatched_user_df

,userID,date,userLocation,userDescription,description,location,id,user_city,user_state,user_city_state,place_city_state,state_US
1,479920728,2017-02-02 11:33:39,where im meant to be,,,where im meant to be,479920728,,WHERE IM MEANT TO BE,,,
2,748510143487279105,2017-02-02 11:33:39,67th Langley,,,67th Langley,748510143487279105,,67TH LANGLEY,,,
4,322642659,2017-02-02 11:33:39,Maryland,Social Justice minded United Methodist Deacon; spouse of an #UMC Elder; mom of an exceptional aspie; lifetime member of the Lyme disease community,Social Justice minded United Methodist Deacon; spouse of an #UMC Elder; mom of an exceptional aspie; lifetime member of the Lyme disease community,Maryland,322642659,,MD,,,
5,34476701,2017-02-02 11:33:39,Illinois,MERICA ! under construction - cars are life,MERICA ! under construction - cars are life,Illinois,34476701,,IL,,,
7,107890250,2017-02-02 11:33:39,LosAngeles-Heidelberg-Caesarea,A good-for-nothing crusty old bugger Editor and biblical scholar with PhD became a buskerhttp://youtu.be/QYxNiKiMwCI,A good-for-nothing crusty old bugger Editor and biblical scholar with PhD became a busker,LosAngeles-Heidelberg-Caesarea,107890250,,LOSANGELES-HEIDELBERG-CAESAREA,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
654542,114966176,2019-01-04 23:03:11,CT/NJ Bay Area,Founder; @fourth_nine | Cover Four Podcast on @iTunes | @UConn alum | Previously: @NinersNation; @BleacherReport & @NBCSAuthentic | https://apple.co/2soCa5l,Founder; | Cover Four Podcast on | alum | Previously: ; & |,CT/NJ Bay Area,114966176,,CT/NJ BAY AREA,,,
654543,2375578866,2019-01-04 23:03:11,America,,,America,2375578866,,AMERICA,,,
654548,134105717,2019-01-04 23:03:12,tuscaloosa,my son; my moon ; and all of my stars. 03-13-18,my son; my moon ; and all of my stars. 03-13-18,tuscaloosa,134105717,,TUSCALOOSA,,,
654549,43212382,2019-01-04 23:03:12,International,Public Relations Professional. 26. #AKA1908,Public Relations Professional. 26. #AKA1908,International,43212382,,INTERNATIONAL,,,


In [907]:
# not_empty_unmatched_user_df.query("user_state !='US' ")
unlocalized_df = not_empty_unmatched_user_df.query("user_state !='US' ")
other_state_idx = unlocalized_df['user_state'].isin(state_abbr_df['Abbreviation'].to_list())
other_state_df = unlocalized_df[other_state_idx]  # .merge(state_abbr_df, left_on='user_state', right_on='Abbreviation')#.sample(10)
unlocalized_df = unlocalized_df[~other_state_idx]
unlocalized_df

print("State user rate:")
len(other_state_df['id'].unique()) / total_user_cnt + len(state_matched_user_df['id'].unique()) / total_user_cnt


State user rate:


0.14768042215557498

In [908]:
# unlocalized_df.sample(10)
print("unlocalized user rate:")
unlocalized_df
len(unlocalized_df['id'].unique()) / total_user_cnt 

unlocalized user rate:


0.32180061501081964

In [898]:
unlocalized_df['location'].value_counts()

Toronto; Ontario            3822
Los Angeles                 2734
New York City               2492
NYC                         2186
Chicago                     2087
                            ... 
Boonton Township            1   
Blue Avenue                 1   
East Moline; Illinois :)    1   
254 TX                      1   
HopkinsDecaturMissouri      1   
Name: location, Length: 105325, dtype: int64

In [897]:
# unlocalized_df.sample(10)
import pprint
# pprint.pprint(unlocalized_df['location'].value_counts().to_dict())
t_df = pd.DataFrame(unlocalized_df['location'].value_counts())
for i, row in t_df.iloc[:20].iterrows():
    print(i, row['location'])

Toronto; Ontario 3822
Los Angeles 2734
New York City 2492
NYC 2186
Chicago 2087
New York State 2026
Toronto 1862
Canada 1729
Monterrey; Nuevo Len 942
Vancouver; British Columbia 930
Earth 911
San Francisco 910
Atlanta 898
Ontario; Canada 859
Ottawa; Ontario 843
Monterrey; Nuevo León 823
Philadelphia 761
Houston 739
Toronto; Canada 670
Seattle 633


In [909]:
stop

NameError: name 'stop' is not defined

In [988]:
# raw_user_df

# Get location name by GPT

In [1074]:
# user_CSV = r'.\user_500.csv'
# user_CSV = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Tweet_geo\usall_1m.csv'
# user_CSV = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Tweet_geo\worldallnogeo_1m.csv'
user_CSV = r'P:\My Drive\worldallnogeo_1m.csv'
# raw_user_df = pd.read_csv(user_CSV, encoding='utf-8', engine='python', error_bad_lines=False)#.fillna("")
raw_user_df = pd.read_csv(user_CSV, engine='c',  encoding='utf-8').fillna("")


# format the column names
raw_user_df['description'] = raw_user_df['userDescription'] 
raw_user_df['location'] = raw_user_df['userLocation'] 
raw_user_df['location'] = raw_user_df['location'].str.replace("\t ", "").replace("\n", "")

raw_user_df['id'] = raw_user_df['userID'] 

used_cols=['id', 'description', 'location']
raw_user_df = raw_user_df.fillna("")
raw_user_df['description'] = raw_user_df['description'].apply(helper.clean_tweet)
raw_user_df['location'] = raw_user_df['location'].apply(helper.clean_tweet)

# raw_user_df = raw_user_df.groupby(['userID', 'description', 'location'], as_index=False)['date'].count()
# raw_user_df = raw_user_df.reset_index()

# raw_user_df['user_city'] = raw_user_df['location'].str.split(';').str[-2].str.strip().str.title()
# raw_user_df['user_state'] = raw_user_df['location'].str.split(';').str[-1].str.strip().str.upper() 
# raw_user_df['user_state'] = raw_user_df['user_state'].replace(state_abbr_dict)
# raw_user_df['user_state'] = raw_user_df['user_state'].replace({"UNITED STATES": "US", "United States of America": "US", "USA": "US"})
# # raw_user_df['user_state'] = raw_user_df['user_state'].replace({"USA": "US"})



# raw_user_df['user_city_state'] = raw_user_df['user_city'] + '; ' + raw_user_df['user_state']

# raw_user_df = raw_user_df.groupby(['userID', 'description', 'location'], as_index=False)['date'].count()

raw_user_df#[used_cols]

,userID,date,userLocation,userDescription,description,location,id
0,805442511762313217,2017-11-30 01:31:00,,🚨,🚨,,805442511762313217
1,352734122,2017-11-30 01:31:00,candler road,look at my feet i put my sneakers in the trunk. 919📍,look at my feet i put my sneakers in the trunk. 919📍,candler road,352734122
2,245840327,2017-11-30 01:31:00,,#1.18.13 @niccca_please #RavensNation #TeamLeBron #StriveForGreatness #OriolesNation Coastal Carolina Alum... Current Towson Tiger '19,#1.18.13 #RavensNation #TeamLeBron #StriveForGreatness #OriolesNation Coastal Carolina Alum... Current Towson Tiger '19,,245840327
3,737410897618997249,2017-11-30 01:31:00,Corpus Christi; TX,Art. Design. Film. Media. Bridging the gap between the real world and the digital one. IG: HiResMediaGroup,Art. Design. Film. Media. Bridging the gap between the real world and the digital one. IG: HiResMediaGroup,Corpus Christi; TX,737410897618997249
4,2894323380,2017-11-30 01:31:00,Sjdm Bulacan,Ig | EGFROMTHEEAST Fb | Ella Gonzaga,Ig | EGFROMTHEEAST Fb | Ella Gonzaga,Sjdm Bulacan,2894323380
...,...,...,...,...,...,...,...
999994,1117591094,2016-05-01 08:39:17,,,,,1117591094
999995,125486926,2016-05-01 08:39:17,japan,T-ARAが大好きでまたツイ始めました＾＾,T-ARAが大好きでまたツイ始めました＾＾,japan,125486926
999996,2251943851,2016-05-01 08:39:17,,Se sei triste; metti un po' più di rossetto e vai all'attacco. Coco Chanel,Se sei triste; metti un po' più di rossetto e vai all'attacco. Coco Chanel,,2251943851
999997,3242681246,2016-05-01 08:39:17,,,,,3242681246


In [1068]:
raw_user_df.query('location == "" and description == "" ')

,index,userID,description,location,date
0,0,,,,649
14,14,1000010965847629824,,,1
17,17,1000012526439030784,,,1
27,27,100001851,,,1
35,35,1000024098867744768,,,1
...,...,...,...,...,...
970754,970754,999905202705649665,,,1
970762,970762,999914410058526722,,,1
970789,970789,99994317,,,1
970799,970799,99995545,,,1


In [1069]:
len(raw_user_df['location'].unique())

315036

In [1070]:
raw_user_df.sample(10)

,index,userID,description,location,date
436175,436175,2904337522,İyilikhane Yetimlerle Dayanışma Derneği gönüllülerinin Twitter hesabıdır. Kurumsal hesap:,,1
391971,391971,2719912355,k.e.s.h.a,,1
737088,737088,707278034672467968,LEIAM AS MINHAS FICS:,✨Jessine best friendship✨,1
316481,316481,237291060,Singer of MEGARA - Alice in Wonderland Lover ;; Zombie Killer - I'm bad with relationships,Madrid,1
273580,273580,2209706005,サブカル好きの紳士 声フェチ 乳、脚、腿、etc…,,1
44246,44246,1086263427339878401,,,1
580124,580124,3878533994,バズリズムライヴ1108✨ 天戸3C ☞☞ CK F1-6 // CKWO♬ trombone / GENERATIONS / EXILE TRIBE / E-girls / サッカー / 内田篤人 / はじめしゃちょー,,1
224341,224341,176844616,ジークいちご帝国,安田王国も州オクラ市風魔区鍋町6-29るひマンション825,1
274681,274681,2214082338,Catch me up on Instagram ; Tumblr Flickr Pinterest,Lagos; Nigeria,1
814470,814470,798696374,,,1


In [1071]:
 

# 

def LLM_clean_home_location(user_df):
    user_df['raw_id'] = user_df['id']
    user_df['id'] = range(1, len(user_df) + 1, 1)


    # model=r"gpt-3.5-turbo-0613"
    # model=r"gpt-3.5-turbo"
    model = "gpt-4"

    tweet_loc = Localization(user_df=user_df,
                            model=model,
                            max_tweet=80,
                            used_cols=['id', 'description', 'location'],
                            )

    print(tweet_loc.prompt_for_user_localization)
    print()
    response_for_user_loc = tweet_loc.get_LLM_user_loc_response() 

    content = helper.extract_content_from_LLM_reply(response_for_user_loc)
    content = content.replace(";;", ";").replace("\n", "")

    # print("content", content)
    if content[-1] == ";":
        content = content[:-1]
    # extracted_location_list = ast.literal_eval(content)
    # extracted_location_list = content.split(";")
    # print("content", content)


    extracted_location_list = helper.location_str_to_list(content)
    extracted_loc_df = pd.DataFrame(extracted_location_list, columns=['id', 'extracted_location'])
    extracted_loc_df['id'] = extracted_loc_df['id'].astype(int)

    if len(user_df) != len(extracted_location_list):
        print("***************  ERROR  *****************")
        print(f"The extracted location count ({len(extracted_location_list)}) is not equal to the input user count ({len(user_df)})!")

    else:
        print(f"The extracted location count ({len(extracted_location_list)}) is  equal to the input user count ({len(user_df)})!")
        # print("Extracted locations:")
        # print(extracted_location_list)

    merged_df = user_df[used_cols].merge(extracted_loc_df, how='right', right_on='id', left_on='id') 
    merged_df['id'] = user_df['raw_id'].to_list()
    
    return merged_df


# user_df = raw_user_df.fillna("").sample(20)  # maximum = 80.

# merged_df =  LLM_clean_home_location(user_df=user_df)
# merged_df

In [1072]:
 # raw_user_df#.groupby(['userID', 'description', 'location'])[''].count()

In [1075]:
import warnings
from IPython.display import clear_output

warnings.filterwarnings('ignore')

sample_size = 80

saved_path = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Tweet_geo\test_results'

# df_count = int(len(raw_user_df) / sample_size)

# raw_user_df = raw_user_df.groupby(['userID', 'description', 'location'], as_index=False).agg(date,count)

# last_pos = 0
start_pos = 1200
stop_pos = 11000
for last_pos in range(start_pos, stop_pos, sample_size):
    try:        
        user_df = raw_user_df.iloc[last_pos: last_pos + sample_size]# maximum = 80.
        print(f"Current rows: {last_pos} / {stop_pos}")
        merged_df =  LLM_clean_home_location(user_df=user_df)
        new_name = os.path.join(saved_path, f"{last_pos}.csv")
        merged_df.to_csv(new_name, index=False)
        clear_output()
    except Exception as e:
        print("Error:", e, last_pos)
        continue
        
        

# Analyze results

In [1104]:
from glob import glob

result_dir = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Tweet_geo\test_results'

CSV_files = glob(os.path.join(result_dir, "*.csv"))

print("Found files:", len(CSV_files))

Found files: 116


In [1105]:
result_df = pd.concat([pd.read_csv(f) for f in CSV_files])
result_df

,id,description,location,extracted_location
0,805442511762313217,🚨,NaN,NaN
1,352734122,look at my feet i put my sneakers in the trunk. 919📍,candler road,"city,Candler,GA,US"
2,245840327,#1.18.13 #RavensNation #TeamLeBron #StriveForGreatness #OriolesNation Coastal Carolina Alum... Current Towson Tiger '19,NaN,"city,Towson,MD,US"
3,737410897618997249,Art. Design. Film. Media. Bridging the gap between the real world and the digital one. IG: HiResMediaGroup,Corpus Christi; TX,"city,Corpus Christi,TX,US"
4,2894323380,Ig | EGFROMTHEEAST Fb | Ella Gonzaga,Sjdm Bulacan,"city,San Jose del Monte,Bulacan,PH"
...,...,...,...,...
75,25264070,DJ; Drag Queen and Adele impersonator from Perth; Australia,Perth; Australia,"city,Perth, AU"
76,464211207,20. fsu.,MD,"subadmin,Maryland, US"
77,15601132,Godfather of the Wright Firm LLP! Jesuit grad; STA Dallas&Jesuit Parent;#Attorney/CPA who loves #estate planning; #probate; tax; litigation & all things #tech!,Dallas,"city,Dallas, TX, US"
78,854777799587975169,정재현 이제노 /2n 나 놀아주셍용,NaN,NaN


In [1127]:

result_df['extracted_location'] = result_df['extracted_location'].fillna("unknown")
result_df['extracted_location'] = result_df['extracted_location'].replace({"state": "subadmin", "province":"subadmin"})
result_df['extracted_location'] = result_df['extracted_location'].str.replace("state", "subadmin").str.replace("province", "subadmin")


result_df['extracted_level_count'] = result_df['extracted_location'].str.split(',').str[0]
result_df.to_csv(r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Tweet_geo\result_merged.csv', index=False)

In [1128]:
len(result_df['id'].unique())

9123

In [1130]:
# result_df['extracted_location']#.replace({"state": "subadmin", "province":"subadmin"})#.query("extracted_level == 'state' ")
result_df.sample(20)[['description', 'location', 'extracted_location', 'extracted_level']]

,description,location,extracted_location,extracted_level
45,NaN,NaN,unknown,unknown
63,4165793757 // sc; maddiieelane // J❤,Ontario; Canada,"subadmin,Ontario,CA",subadmin
17,NY. Nike. Kobe. Kanye.,NaN,unknown,unknown
62,Portuguese 🇵🇹 THS💛🐶💙 15👀,NaN,unknown,unknown
28,NaN,CA,"subadmin,CA, US",subadmin
21,Ainda que eu ande no vale das sombras da morte não temerei mal algum . O senhor e meu pastor e nada me faltará.,Rio de Janeiro; Brasil,"city,Rio de Janeiro, BR",city
69,NaN,NaN,unknown,unknown
60,Atleta do Progresso F.C ⚫️🔴 snap Kauaescobiris03 👻 Rio Grande rs 🏠,NaN,unknown,unknown
78,NaN,NaN,unknown,unknown
5,NaN,NaN,unknown,unknown


In [1118]:


analyzed_df = pd.DataFrame(result_df['extracted_level_count'].value_counts())# / len(result_df)
total_tweet = len(result_df)
analyzed_df['extracted_level_rate'] = analyzed_df['extracted_level_count'] / total_tweet
analyzed_df

,extracted_level_count,extracted_level_rate
unknown,5026,0.545711
city,2882,0.312921
country,705,0.076547
subadmin,572,0.062106
lonlat,16,0.001737
region,2,0.000217
us_subadmin,1,0.000109
planet,1,0.000109
neighborhood,1,0.000109
Subadmin,1,0.000109


# Geocoding API

In [1168]:
import pandas as pd
import requests
import json
from pprint import pprint
import pandas as pd


result_df = pd.read_csv(r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Tweet_geo\result_merged.csv' )


,id,description,location,extracted_location,extracted_level,extracted_level_count
0,805442511762313217,🚨,NaN,unknown,unknown,unknown
1,352734122,look at my feet i put my sneakers in the trunk. 919📍,candler road,"city,Candler,GA,US",city,city
2,245840327,#1.18.13 #RavensNation #TeamLeBron #StriveForGreatness #OriolesNation Coastal Carolina Alum... Current Towson Tiger '19,NaN,"city,Towson,MD,US",city,city
3,737410897618997249,Art. Design. Film. Media. Bridging the gap between the real world and the digital one. IG: HiResMediaGroup,Corpus Christi; TX,"city,Corpus Christi,TX,US",city,city
4,2894323380,Ig | EGFROMTHEEAST Fb | Ella Gonzaga,Sjdm Bulacan,"city,San Jose del Monte,Bulacan,PH",city,city
...,...,...,...,...,...,...
9205,25264070,DJ; Drag Queen and Adele impersonator from Perth; Australia,Perth; Australia,"city,Perth, AU",city,city
9206,464211207,20. fsu.,MD,"subadmin,Maryland, US",subadmin,subadmin
9207,15601132,Godfather of the Wright Firm LLP! Jesuit grad; STA Dallas&Jesuit Parent;#Attorney/CPA who loves #estate planning; #probate; tax; litigation & all things #tech!,Dallas,"city,Dallas, TX, US",city,city
9208,854777799587975169,정재현 이제노 /2n 나 놀아주셍용,NaN,unknown,unknown,unknown


In [1167]:

import requests
import json
from pprint import pprint
import pandas as pd

def geocode(address):
    base_url = "https://nominatim.openstreetmap.org/search"
    parameters = {
        'q': address,
        'format': 'json',
        'accept-language': 'en',
        'limit': 1,
        'addressdetails': 1,
        # 'polygon_geojson': 1,
    }
    response = requests.get(base_url, params=parameters)
    
    if response.status_code == 200:
        # Extract the first result. If there are no results, it returns None
        data = response.json()
        if data:
            return data 
        else:
            return None
    else:
        raise Exception(f"Request failed with status {response.status_code}")

# # Testing the function
# data = geocode('Davao City	')
# pprint(data)


In [ ]:
import time
result_df = result_df.fillna("")# .query('location != "" ').sample(20)
result_df['OSM_location'] = ''
for i, row in result_df.iterrows():
    
    location = row['location']
    print(f"Processed: {i + 1} / {len(result_df)}")
    if location == "":
        continue
        
    
        
    print(location, '\n')
    data = geocode(location)
    pprint(data)
    time.sleep(0.3)
    print()
    # clear_output()
    
    result_df.at[i, 'OSM_location'] = data
    
result_df.to_csv(r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Tweet_geo\result_merged_OSM.csv', index=False)

Processed: 1 / 9210
Processed: 2 / 9210
candler road 

[{'address': {'ISO3166-2-lvl4': 'US-GA',
              'country': 'United States',
              'country_code': 'us',
              'county': 'DeKalb County',
              'postcode': '30034',
              'road': 'Candler Road',
              'state': 'Georgia',
              'town': 'Panthersville'},
  'boundingbox': ['33.7050482', '33.706989', '-84.2720287', '-84.2713315'],
  'class': 'highway',
  'display_name': 'Candler Road, Panthersville, DeKalb County, Georgia, 30034, '
                  'United States',
  'importance': 0.30000999999999994,
  'lat': '33.7060635',
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. '
             'https://osm.org/copyright',
  'lon': '-84.2717456',
  'osm_id': 80428417,
  'osm_type': 'way',
  'place_id': 123945249,
  'type': 'secondary'}]

Processed: 3 / 9210
Processed: 4 / 9210
Corpus Christi; TX 

[{'address': {'ISO3166-2-lvl4': 'US-TX',
              'city': 'Corpus Christi',
  